1 Year Data

1. 5args
2. 7args
3. 加大盤
4. 加買超賣超


*   5/20
*   10/20
*   20/10







Libraries

In [6]:
import pandas as pd
import numpy as np
import joblib
import math
import os
from tqdm import tqdm
from datetime import datetime, timedelta
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Dropout
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline
from google.colab import drive

drive.mount('/content/drive')

FIVE_ARGS = ['Open', 'Close', 'High', 'Low', 'Volume']
SIX_ARGS = ['Open', 'Close', 'High', 'Low', 'Volume', 'II']
SEVEN_ARGS = ['Open', 'Close', 'High', 'Low', 'Volume', 'SMA', 'BIAS']
EIGHT_ARGS = ['Open', 'Close', 'High', 'Low', 'Volume', 'SMA', 'BIAS', 'II']

home = '/content/drive/MyDrive/Stock-Market-Price-Prediction/'
input_dir = '0050-1224'
params_dir = 'final_model_params'
predict_start_day = datetime(2023, 12, 18)
output_dir = f'predict_{predict_start_day.year}{predict_start_day.month}{predict_start_day.day}'

SMA_num = 20
predict_num = 5

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Read final model csv and initialize directory


In [7]:
company_list = ['智邦', '中信金', '統一超', '玉山金', '鴻海', '日月光投控', '亞德客-KY', '瑞昱', '中鋼', '聯發科', '大立光', '南亞科', '台泥', '緯創', '兆豐金', '合庫金', '和碩', '第一金', '遠傳', '豐泰', '富邦金', '台積電', '永豐金', '元大金', '台塑', '欣興', '開發金', '和泰車', '統一', '長榮', '緯穎', '廣達', '上海商銀', '台塑化', '南亞', '國巨', '中租-KY', '華南金', '彰銀', '台新金', '國泰金', '研華', '華碩', '聯詠', '台化', '台灣大', '中華電', '光寶科', '聯電', '台達電']
company_list = company_list[:]

final_model = pd.read_csv(home + 'final_model.csv')
final_model.columns = ['company', 'input', 'args', 'mse', 'model', 'days' , 'comment1', 'start', 'comment2']
final_model = final_model.drop(['mse', 'comment1', 'comment2'], axis=1).loc[1:, :].reset_index(drop=True)
# print(final_model)

if output_dir not in os.listdir(home):
  os.mkdir(home+output_dir)

for company in company_list:
  if company not in os.listdir(home+output_dir):
    os.mkdir(home+output_dir+'/'+company)
  for f in os.listdir(home+output_dir+'/'+company):
    os.remove(home+output_dir+'/'+company+'/'+f)

output_path = home + output_dir + '/'

Input CSV File

In [8]:
company_df_dict = {}
for company in company_list:
  # print(company)

  df = pd.read_csv(home+input_dir+'/'+company+'/K_Chart.csv')
  df = df.iloc[2:, :16]
  df = df.drop(['漲跌','漲跌.1','振幅','Unnamed: 9','Unnamed: 10','Unnamed: 11','法人買賣超(張)','Unnamed: 13','Unnamed: 14'],axis=1)
  df.columns = ['Date','Open','High','Low','Close','Volume','II']
  df['Date'] = df['Date'].apply(lambda x: '2023年'+str(x))
  df['Date'] = pd.to_datetime(df['Date'].values, format='%Y年%m月%d日')
  df['Volume'] = df['Volume'].apply(lambda x: float(x.replace(',','')))
  df['II'] = df['II'].apply(lambda x: float(str(x).replace(',','')))
  df = df.iloc[::-1].reset_index(drop=True)

  if int(final_model.loc[final_model['company'] == company, 'args'].values[0]) == 5 or int(final_model.loc[final_model['company'] == company, 'args'].values[0]) == 7:
    df = df.drop(['II'], axis=1)
  if int(final_model.loc[final_model['company'] == company, 'args'].values[0]) >= 7:
    SMA = []
    BIAS = []
    for i in range(len(df)):
      if i < SMA_num - 1:
        SMA.append(0)
        BIAS.append(0)
      else:
        SMA.append(df.loc[i+1-SMA_num:i+1, 'Open'].mean())
        BIAS.append((df.loc[i, 'Open'] - SMA[i]) / SMA[i])
    df['SMA'] = SMA
    df['BIAS'] = BIAS
    df = df.loc[SMA_num:].reset_index(drop=True)

  # print(df.head(10))

  company_df_dict[company] = df

Functions

In [9]:
def Get_Dataset(df, input_num):
  X_pred = []

  for predict_pre in range(len(df)):
    if predict_pre == len(df) - 1:
      break
    if df.loc[predict_pre, 'Date'] < predict_start_day and df.loc[predict_pre+1, 'Date'] >= predict_start_day:
      break

  start_index = predict_pre - input_num - predict_num + 2
  df = df.drop(['Date'],axis=1)

  for i in range(start_index, start_index+predict_num):
    X_pred.append(df.loc[i:i+input_num-1].values)

  return np.array(X_pred)

def get_statistic(df_prediction):
  df_prediction = df_prediction.reset_index(drop=True)
  ae, se = 0.0, 0.0
  for i in range(len(df_prediction)):
    ae += df_prediction.loc[i,'Open'] - df_prediction.loc[i,'Prediction'] if df_prediction.loc[i,'Open'] > df_prediction.loc[i,'Prediction'] else  df_prediction.loc[i,'Prediction'] - df_prediction.loc[i,'Open']
    se += (df_prediction.loc[i,'Open'] - df_prediction.loc[i,'Prediction']) ** 2
  return ae / len(df_prediction), se / len(df_prediction)

Main

In [10]:
for company, df_comp in tqdm(company_df_dict.items()):
  print('\n'+company+'\n')

  input_num = int(final_model.loc[final_model['company'] == company, 'input'])

  # load scaler and apply
  for col in df_comp.columns:
    if col == 'Date':
      continue
    scaler = joblib.load(home+params_dir+'/'+company+f'/scaler_{col}.z')
    df_comp[col] = scaler.fit_transform(df_comp.filter([col]).values)

  # load model and make prediction
  X_pred = Get_Dataset(df_comp, input_num)
  model = load_model(home+params_dir+'/'+company+'/model.keras')
  predictions = model.predict(X_pred)
  scaler_open = joblib.load(home+params_dir+'/'+company+'/scaler_Open.z')
  predictions = scaler_open.inverse_transform(predictions)

  # save output into csv and png
  date_list = []
  i = 0
  for _ in range(predict_num):
    while (predict_start_day + timedelta(i)).weekday() >= 5:
      i += 1
    date_list.append((predict_start_day + timedelta(i)).strftime('%Y-%m-%d'))
    i += 1
  df_output = pd.DataFrame({'Date':date_list, 'Open':predictions.reshape(predictions.shape[0])})
  df_output.to_csv(output_path+company+'/prediction.csv')

  df_comp['Open'] = scaler_open.inverse_transform(df_comp.filter(['Open']).values)
  df_comp['Prediction'] = np.zeros(len(df_comp))

  start_index = df_comp.index[df_comp['Date'] == pd.to_datetime(df_output.loc[0, 'Date'], format='%Y-%m-%d')].tolist()[0]
  for i in range(predict_num):
    df_comp.loc[start_index+i, 'Prediction'] = df_output.loc[i, 'Open']

  df_prediction = df_comp[start_index:start_index+predict_num]

  mae, mse = get_statistic(df_prediction)

  df_comp = df_comp.set_index('Date',drop=True)
  df_prediction = df_prediction.set_index('Date', drop=True)

  plt.style.use('dark_background')
  plt.figure(figsize=(16,8))
  plt.title(f'{company} Price Line Chart')
  plt.xlabel('Date', fontsize=18)
  plt.ylabel('Open Price', fontsize=18)
  plt.plot(df_comp['Open'], color='red')
  plt.plot(df_prediction['Prediction'], color='white')
  plt.figtext(0.2,0.8,f'MAE:{round(mae,3)}\nMSE:{round(mse,3)}', color='white', fontsize=18)
  plt.legend(['True', 'Predict'], loc='lower left')
  plt.savefig(output_path+company+'/plot.png')
  # plt.show()

Output hidden; open in https://colab.research.google.com to view.